In [1]:
# IMPORTING PACKAGES

import pandas as pd 
import matplotlib.pyplot as plt 
import requests
import math
from termcolor import colored as cl 
import numpy as np
from secrets import IEX_CLOUD_API_TOKEN


plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 8)


In [2]:
all_stocks = pd.read_csv("sp_500_stocks.csv")
all_stocks = pd.DataFrame(all_stocks)
print(all_stocks)
i = 0
for symbol in all_stocks['Ticker']:
    if symbol == 'ETFC':
        print (i)
    i = i + 1

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]
165


In [12]:
def sma(data, n):
        sma = data.rolling(window = n).mean()
        return pd.DataFrame(sma)
    
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

# get data
def get_historic_data(symbol):
    iex_api_key = 'Tpk_1a21b6507ed04ea288d53fc96d23d8d6'
    try:
        api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6'
        df = requests.get(api_url)
        print(df.url)
        print(df.status_code)
        df = df.json()
        with open("json_info.txt", 'w+') as f:
            f.write(df)
    except:
        return pd.DataFrame()
    
    date = []
    opened = []
    high = []
    low = []
    close = []
    volume = []
    ticker = []
    
    for i in df:
        print(i)
        for j in df[i]['quote']:
            print(j)
            date.append(df[i][j]['date'])
            opened.append(df[i][j]['open'])
            high.append(df[i][j]['high'])
            low.append(df[i][j]['low'])
            close.append(df[i][j]['close'])
            volume.append(df[i][j]['volume'])
            ticker.append(df[i][0]['symbol'])
        
    #creating dataframes of each list and concatinating them 
    date_df = pd.DataFrame(date).rename(columns = {0:'date'})
    open_df = pd.DataFrame(opened).rename(columns = {0:'open'})
    high_df = pd.DataFrame(high).rename(columns = {0:'high'})
    low_df = pd.DataFrame(low).rename(columns = {0:'low'})
    close_df = pd.DataFrame(close).rename(columns = {0:'close'})
    volume_df = pd.DataFrame(volume).rename(columns = {0: 'volume'})
    ticker_df = pd.DataFrame(ticker).rename(columns = {0: 'ticker'})
    
    frames = [date_df, open_df, high_df, low_df, close_df,volume_df]
    df = pd.concat(frames, axis = 1, join = 'inner')
    
    return df


# CREATING SMA TRADING STRATEGY

def implement_sma_strategy(data, short_window, long_window):
    sma1 = short_window
    sma2 = long_window
    buy_price = []
    sell_price = []
    sma_signal = []
    signal = 0
    
    for i in range(len(data)):
        if sma1[i] > sma2[i]:
            if signal != 1:
                buy_price.append(data[i])
                sell_price.append(np.nan)
                signal = 1
                sma_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                sma_signal.append(0)
        elif sma2[i] > sma1[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(data[i])
                signal = -1
                sma_signal.append(-1)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                sma_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            sma_signal.append(0)
            
    return buy_price, sell_price, sma_signal

# chunked_stocks = list(chunks(all_stocks['Ticker'], 25))
# symbol_strings = []
# for i in range(0, len(chunked_stocks)):
#     symbol_strings.append(','.join(chunked_stocks[i]))
    
    
# for symbol_string in symbol_strings:
# #     print(symbol_strings)
#     data = get_historic_data(symbol_string)
#     for symbol in symbol_string.split(','):
#         final_dataframe = final_dataframe.append(
#                                         pd.Series([symbol, 
#                                                    data[symbol]['quote']['latestPrice'], 
#                                                    data[symbol]['quote']['marketCap'], 
#                                                    'N/A'], 
#                                                   index = my_columns), 
#                                         ignore_index = True)
        

In [14]:

all_results = {}

for symbol in all_stocks['Ticker']:
    print(symbol)
    
    stock = get_historic_data(symbol)
    
    if stock.empty:
        continue
    stock = stock.set_index('date')
    stock = stock[stock.index >= '2018-01-01']
    stock.index = pd.to_datetime(stock.index)
    
    print(stock.iloc[-1]['close'],stock.iloc[0]['close'])

    n = [20, 50, 100, 200]
    for i in n:
        stock[f'sma_{i}'] = sma(stock['close'], i)

    sma_20 = stock['sma_20']
    sma_50 = stock['sma_50']

    buy_price, sell_price, signal = implement_sma_strategy(stock['close'], sma_20, sma_50)
    
    position = []
    for i in range(len(signal)):
        if signal[i] > 1:
            position.append(0)
        else:
            position.append(1)
            
    for i in range(len(stock['close'])):
        if signal[i] == 1:
            position[i] = 1
        elif signal[i] == -1:
            position[i] = 0
        else:
            position[i] = position[i-1]

    sma_20 = pd.DataFrame(sma_20).rename(columns = {0:'sma_20'})
    sma_50 = pd.DataFrame(sma_50).rename(columns = {0:'sma_50'})
    signal = pd.DataFrame(signal).rename(columns = {0:'sma_signal'}).set_index(stock.index)
    position = pd.DataFrame(position).rename(columns = {0:'sma_position'}).set_index(stock.index)
    frames = [sma_20, sma_50, signal, position]
    strategy = pd.concat(frames, join = 'inner', axis = 1)
    strategy = strategy.reset_index().drop('date', axis = 1)

            
    strategy['differences'] = pd.DataFrame(np.diff(stock['close']))
    stock_ret = pd.DataFrame(np.diff(stock['close'])).rename(columns = {0:'returns'})
    sma_strategy_ret = []


    # BACKTESTING THE STRAGEGY

    for i in range(len(stock_ret)):
        try:
            returns = stock_ret['returns'][i] * strategy['sma_position'][i]
            sma_strategy_ret.append(returns)
        except:
            pass

    #creating dataframe of a normal list    
    sma_strategy_ret_df = pd.DataFrame(sma_strategy_ret).rename(columns = {0:'sma_returns'})

    investment_value = 100000
    number_of_stocks = math.floor(investment_value/stock['close'][1])
    sma_investment_ret = []

    for i in range(len(sma_strategy_ret_df['sma_returns'])):
        returns = number_of_stocks * sma_strategy_ret_df['sma_returns'][i]
        sma_investment_ret.append(returns)


    sma_investment_ret_df = pd.DataFrame(sma_investment_ret).rename(columns = {0:'investment_returns'})
    total_investment_ret = round(sum(sma_investment_ret_df['investment_returns']), 2)
    
    hold_value = number_of_stocks * (stock.iloc[-1]['close'] - stock.iloc[0]['close'])

    with open("trading_results2.txt", "a") as f:
        file_data = [str(symbol), str(hold_value),  str(total_investment_ret)]
        for piece in file_data:
            f.write(piece + '\n')
        
    
    all_results[symbol] = total_investment_ret
    
print(all_results)

A
https://sandbox.iexapis.com/stable/stock/A/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
AAL
https://sandbox.iexapis.com/stable/stock/AAL/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
AAP
https://sandbox.iexapis.com/stable/stock/AAP/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
AAPL
https://sandbox.iexapis.com/stable/stock/AAPL/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ABBV
https://sandbox.iexapis.com/stable/stock/ABBV/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ABC
https://sandbox.iexapis.com/stable/stock/ABC/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ABMD
https://sandbox.iexapis.com/stable/stock/ABMD/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ABT
https://sandbox.iexapis.com/stable/stock/ABT/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ACN
https://sandbox.iexapis.com/stable/stock/ACN/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ADBE
https://sandbox.iexapis.com/stable/stock/ADBE/chart?token=Tpk_1a21b6507ed04

https://sandbox.iexapis.com/stable/stock/CB/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
CBOE
https://sandbox.iexapis.com/stable/stock/CBOE/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
CBRE
https://sandbox.iexapis.com/stable/stock/CBRE/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
CCI
https://sandbox.iexapis.com/stable/stock/CCI/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
CCL
https://sandbox.iexapis.com/stable/stock/CCL/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
CDNS
https://sandbox.iexapis.com/stable/stock/CDNS/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
CDW
https://sandbox.iexapis.com/stable/stock/CDW/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
CE
https://sandbox.iexapis.com/stable/stock/CE/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
CERN
https://sandbox.iexapis.com/stable/stock/CERN/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
CF
https://sandbox.iexapis.com/stable/stock/CF/chart?token=Tpk_1a21b6507ed04ea288

https://sandbox.iexapis.com/stable/stock/EQIX/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
EQR
https://sandbox.iexapis.com/stable/stock/EQR/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ES
https://sandbox.iexapis.com/stable/stock/ES/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ESS
https://sandbox.iexapis.com/stable/stock/ESS/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ETFC
https://sandbox.iexapis.com/stable/stock/ETFC/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ETN
https://sandbox.iexapis.com/stable/stock/ETN/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ETR
https://sandbox.iexapis.com/stable/stock/ETR/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
EVRG
https://sandbox.iexapis.com/stable/stock/EVRG/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
EW
https://sandbox.iexapis.com/stable/stock/EW/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
EXC
https://sandbox.iexapis.com/stable/stock/EXC/chart?token=Tpk_1a21b6507ed04ea288d5

https://sandbox.iexapis.com/stable/stock/INTC/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
INTU
https://sandbox.iexapis.com/stable/stock/INTU/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
IP
https://sandbox.iexapis.com/stable/stock/IP/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
IPG
https://sandbox.iexapis.com/stable/stock/IPG/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
IPGP
https://sandbox.iexapis.com/stable/stock/IPGP/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
IQV
https://sandbox.iexapis.com/stable/stock/IQV/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
IR
https://sandbox.iexapis.com/stable/stock/IR/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
IRM
https://sandbox.iexapis.com/stable/stock/IRM/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
ISRG
https://sandbox.iexapis.com/stable/stock/ISRG/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
IT
https://sandbox.iexapis.com/stable/stock/IT/chart?token=Tpk_1a21b6507ed04ea288d5

https://sandbox.iexapis.com/stable/stock/MTD/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
MU
https://sandbox.iexapis.com/stable/stock/MU/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
MXIM
https://sandbox.iexapis.com/stable/stock/MXIM/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
MYL
https://sandbox.iexapis.com/stable/stock/MYL/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
NBL
https://sandbox.iexapis.com/stable/stock/NBL/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
NCLH
https://sandbox.iexapis.com/stable/stock/NCLH/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
NDAQ
https://sandbox.iexapis.com/stable/stock/NDAQ/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
NEE
https://sandbox.iexapis.com/stable/stock/NEE/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
NEM
https://sandbox.iexapis.com/stable/stock/NEM/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
NFLX
https://sandbox.iexapis.com/stable/stock/NFLX/chart?token=Tpk_1a21b6507ed04ea

https://sandbox.iexapis.com/stable/stock/RTX/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
SBAC
https://sandbox.iexapis.com/stable/stock/SBAC/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
SBUX
https://sandbox.iexapis.com/stable/stock/SBUX/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
SCHW
https://sandbox.iexapis.com/stable/stock/SCHW/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
SEE
https://sandbox.iexapis.com/stable/stock/SEE/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
SHW
https://sandbox.iexapis.com/stable/stock/SHW/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
SIVB
https://sandbox.iexapis.com/stable/stock/SIVB/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
SJM
https://sandbox.iexapis.com/stable/stock/SJM/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
SLB
https://sandbox.iexapis.com/stable/stock/SLB/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
SLG
https://sandbox.iexapis.com/stable/stock/SLG/chart?token=Tpk_1a21b6507ed04

https://sandbox.iexapis.com/stable/stock/WM/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
WMB
https://sandbox.iexapis.com/stable/stock/WMB/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
WMT
https://sandbox.iexapis.com/stable/stock/WMT/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
WRB
https://sandbox.iexapis.com/stable/stock/WRB/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
WRK
https://sandbox.iexapis.com/stable/stock/WRK/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
WST
https://sandbox.iexapis.com/stable/stock/WST/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
WU
https://sandbox.iexapis.com/stable/stock/WU/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
WY
https://sandbox.iexapis.com/stable/stock/WY/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
WYNN
https://sandbox.iexapis.com/stable/stock/WYNN/chart?token=Tpk_1a21b6507ed04ea288d53fc96d23d8d6
200
XEL
https://sandbox.iexapis.com/stable/stock/XEL/chart?token=Tpk_1a21b6507ed04ea288d53fc9